In [ ]:
#Устанавливаем и скачиваем необходимое
!pip install ultralytics
!pip install filterpy
!git clone https://github.com/noahcao/OC_SORT.git

In [ ]:
# Подключаем библиотеки
from ultralytics import YOLO
# Переходим в скачанный каталог
%cd OC_SORT
from trackers.ocsort_tracker.ocsort import OCSort
import numpy as np
import pandas as pd
from IPython.display import clear_output
import cv2

/content/OC_SORT


In [ ]:
# Настройки взяты из args.py
tracker = OCSort(det_thresh=0.6, iou_threshold=0.3, use_byte=False)
# Предобученная модель
model = YOLO("yolov8n.pt")
# Добавьте свое видео
video = '/content/1.mp4'
# Сюда будем накапливать данные
people = np.empty((0, 5))
# Запишем результат в видео
out = cv2.VideoWriter('/content/out.avi',cv2.VideoWriter_fourcc(*'XVID'), 1,
                      (640, 640))
# Обнаружение только людей
for result in model.predict(video, single_cls=True, classes=0):
  frame = result.orig_img
  orig_shp = result.orig_shape
  bbox = result.cpu().boxes.boxes.numpy()
  tracks = tracker.update(bbox[:, :-1], orig_shp, orig_shp)
  if len(tracks) > 0:
    people = np.vstack((people, tracks))
  for t in tracks:
    # Добавим рамки
    cv2.rectangle(frame, (int(t[0]), int(t[1])), (int(t[2]), int(t[3])),
                  (0, 255, 0), 1)
    # Добавим надпись в виде идентификатора объекта
    cv2.putText(frame, str(int(t[4])), (int(t[0]), int(t[1] + 15)),
                cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
  out.write(frame)
out.release()
clear_output()
df = pd.DataFrame(people, columns=['x1', 'y1', 'x2', 'y2', 'id'])
pd.options.mode.chained_assignment = None
# Будем формировать все возможные варианты
df.loc[:, 'last_x1'] = df.x1
df.loc[:, 'last_y1'] = df.y1
df.loc[:, 'last_x2'] = df.x2
df.loc[:, 'last_y2'] = df.y2
df.loc[:, 'min_x1'] = df.x1
df.loc[:, 'max_x1'] = df.x1
df.loc[:, 'min_y1'] = df.y1
df.loc[:, 'max_y1'] = df.y1
df.loc[:, 'min_x2'] = df.x2
df.loc[:, 'max_x2'] = df.x2
df.loc[:, 'min_y2'] = df.y2
df.loc[:, 'max_y2'] = df.y2
agg_func = {'x1': 'first', 'last_x1': 'last', 'min_x1': 'min', 'max_x1': 'max',
            'y1': 'first', 'last_y1': 'last', 'min_y1': 'min', 'max_y1': 'max',
            'x2': 'first', 'last_x2': 'last', 'min_x2': 'min', 'max_x2': 'max',
            'y2': 'first', 'last_y2': 'last', 'min_y2': 'min', 'max_y2': 'max'}
df1 = df.groupby('id').agg(agg_func)
df1.head(20)

,x1,last_x1,min_x1,max_x1,y1,last_y1,min_y1,max_y1,x2,last_x2,min_x2,max_x2,y2,last_y2,min_y2,max_y2
id,,,,,,,,,,,,,,,,
1.0,492.718872,495.938934,484.539429,506.380676,261.310486,283.647003,261.310486,287.947449,639.073242,638.653320,626.620239,639.410889,474.510193,495.509552,472.934692,496.826904
2.0,239.292526,341.327209,239.292526,341.330200,144.672852,281.916443,144.672852,281.957581,292.474060,558.999451,292.474060,559.212891,337.883240,498.055603,337.883240,498.079346
3.0,239.907410,288.090363,237.197311,296.204224,147.976746,233.289795,145.672668,233.289795,289.924988,483.810333,286.127014,483.824860,322.615204,490.601746,322.615204,492.058990
4.0,241.504730,291.819702,240.968887,298.657837,172.342712,269.114868,170.722748,269.114868,318.626312,475.772583,318.595276,475.772583,443.190247,497.490967,364.275970,497.490967
5.0,496.428436,487.312744,480.082642,496.428436,285.091797,239.182770,239.182770,285.091797,633.128174,622.791260,622.784302,639.317871,494.613403,489.528778,489.475525,501.527466


Совсем другая картина. 1 и 5 - охранник. 2, 3, 4 проходят от дальнего края к ближнему (входящие). По значительному вертикальному перемещению их можно определить как входящих.